In [4]:
import pandas as pd
import loldata_eda
import requests
import json
import time
import pandas as pd
from tqdm import tqdm
import numpy as np
from random import randint

In [3]:
import getLoLData2
api_key = "RGAPI-9a2a2c80-d6d6-43ae-9093-86545c40e405"

In [4]:
getLoLData2.getTimelineData(api_key, 6,10, count=10)

no match input.
10
downloading summonerID!
summonerId downloaded!
downloading puuid!


  9%|▉         | 19/205 [00:05<00:55,  3.38it/s]


KeyboardInterrupt: 

In [9]:
matchId = ['KR_6416085964', 'KR_6416039783', 'KR_6416021626', 'KR_6415864679', 'KR_6415827891']

tier input ignored.
downloading TimelineData!


100%|██████████| 5/5 [00:03<00:00,  1.60it/s]

TimelineData downloaded!


,gameId,blueWins,blueTotalGolds,blueCurrentGolds,blueTotalLevel,blueAvgLevel,blueTotalMinionKills,blueTotalJungleMinionKills,blueTotalDamageToChampion,blueFirstBlood,...,redTowerKills,redMidTowerKills,redTopTowerKills,redBotTowerKills,redInhibitor,redFirstDragon,redDragonType,redDragon,redRiftHeralds,matchMin
0,KR_6416085964,False,11882,1857,26,6.5,170,48,10823,1,...,0,0,0,0,0,1,EARTH_DRAGON,1,0,29
0,KR_6416039783,False,12048,2593,27,6.75,186,40,11442,0,...,0,0,0,0,0,0,None,0,0,28
0,KR_6415864679,False,12203,2497,26,6.5,188,44,9965,0,...,0,0,0,0,0,0,None,0,0,30
0,KR_6415827891,True,14235,6160,29,7.25,199,56,11877,1,...,0,0,0,0,0,0,None,0,0,31
